# **TDE 02: APACHE SPARK**
### **Nomes:** Marcella Resende, João Xavier, Sofia Castilho, Bruno Imai

In [1]:
!pip install -q findspark
!pip install -q pyspark

     |████████████████████████████████| 281.4 MB 39 kB/s 
     |████████████████████████████████| 199 kB 57.2 MB/s 


In [2]:
import findspark
findspark.init()

from pyspark import SparkConf, SparkContext, SparkFiles

conf = SparkConf().setAppName("tdeApacheSpark").setMaster("local[*]")

sc = SparkContext.getOrCreate(conf = conf)


rdd = sc.textFile("/content/transactions_menor.csv")

In [ ]:
#Pega a primeira linha - cabeçalho
cabecalho = rdd.first()

In [ ]:
#Remove o cabeçalho
rdd  = rdd.filter(lambda c: c != cabecalho)

In [6]:
#Index das colunas na base de dados
pais = 0
ano = 1
commodity_code = 2
commodity = 3
flow = 4
trade_usd = 5
quantity_name = 7
quantity = 8

# **Questão 1**
### Número de transações envolvendo o Brasil

In [ ]:
#Só resulta onde país possui 'Brazil'
rdd_brasil = rdd.filter(lambda x: x.split(';')[pais] == 'Brazil')

#Realiza a função map e retorna como chave(Brasil, 1)
rdd_transacoes_brasil = rdd_brasil.map(lambda x: (x.split(';')[pais],1))

#Soma as ocorrências encontradas pela chave através da função countByKey
rdd_transacoes_brasil.countByKey()

defaultdict(int, {'Brazil': 54762})

In [ ]:
#Retorna como chave(Brasil, 1)
def map_brasil(linha):
  return ("Brasil", 1)

#Soma as ocorrências encontradas pela chave
def reduce_brasil(a,b):
  return a + b

#Realiza o map através da função 'map_brasil' e em seguida realiza o reduce pela chave através da função reduce_brasil.
#Posteriormente, salva os resultados em um arquivo txt
def transacoes_brasil(con):
  map_resultado = rdd_brasil.map(map_brasil)
  numero_transacoes_brasil = map_resultado.reduceByKey(reduce_brasil)
  numero_transacoes_brasil.saveAsTextFile("/content/output/Q1")

In [ ]:
#chama a função transacoes_brasil com a base de dados
transacoes_brasil(sc)

#Questão 2
### Número de transações por ano

In [ ]:
#Realiza a função map e retorna o ano como chave e 1 como valor
rdd_ano = rdd.map(lambda x: (x.split(';')[ano],1))

#Soma as ocorrências encontradas pela chave através da função countByKey e ordena com a função sorted
sorted(rdd_ano.countByKey().items())

[('1988', 8568),
 ('1989', 16916),
 ('1990', 18188),
 ('1991', 22652),
 ('1992', 32860),
 ('1993', 42031),
 ('1994', 54143),
 ('1995', 63875),
 ('1996', 70390),
 ('1997', 77876),
 ('1998', 81117),
 ('1999', 86191),
 ('2000', 97572),
 ('2001', 97240),
 ('2002', 97202),
 ('2003', 101125),
 ('2004', 103671),
 ('2005', 105913),
 ('2006', 109186),
 ('2007', 105891),
 ('2008', 102680),
 ('2009', 105686),
 ('2010', 105407),
 ('2011', 105486),
 ('2012', 102989),
 ('2013', 104734),
 ('2014', 103231),
 ('2015', 101973),
 ('2016', 82277)]

In [ ]:
#Divide a base de dados através do split tendo como parâmetro ';', em seguida retorna como chave o ano e como valor 1
def map_ano(linha):
  colunas = linha.split(";")
  return (colunas[ano], 1) 

#Soma as ocorrências por chave
def reduce_ano(a,b):
  return a + b

#Realiza o map através da função 'map_ano' e em seguida realiza o reduce pela chave através da função reduce_ano. 
#Posteriormente, salva os resultados em um arquivo txt
def transacoes_ano(con):
  map_resultado = rdd.map(map_ano)
  numero_transacoes_ano = map_resultado.reduceByKey(reduce_ano)
  ##numero = numero_transacoes_ano.sortBy(lambda x: x[1],True)
  numero_transacoes_ano.saveAsTextFile("/content/output/Q2")

In [ ]:
#chama a função transacoes_ano com a base de dados
transacoes_ano(sc)

# Questão 3
### Número de transações por tipo de fluxo e por ano

In [ ]:
#Realiza a função map e retorna o tipo de fluxo e o ano como chave e 1 como valor
rdd_tipo_ano = rdd.map(lambda x: ((x.split(";")[flow], x.split(";")[ano]), 1))

#Soma as ocorrências encontradas pela chave através da função countByKey e ordena com a função sorted
sorted(rdd_tipo_ano.countByKey().items())

[(('Export', '1988'), 10),
 (('Export', '1989'), 12),
 (('Export', '1990'), 7),
 (('Export', '1991'), 9),
 (('Export', '1992'), 19),
 (('Export', '1993'), 22),
 (('Export', '1994'), 30),
 (('Export', '1995'), 33),
 (('Export', '1996'), 32),
 (('Export', '1997'), 41),
 (('Export', '1998'), 32),
 (('Export', '1999'), 48),
 (('Export', '2000'), 58),
 (('Export', '2001'), 46),
 (('Export', '2002'), 60),
 (('Export', '2003'), 49),
 (('Export', '2004'), 47),
 (('Export', '2005'), 50),
 (('Export', '2006'), 47),
 (('Export', '2007'), 55),
 (('Export', '2008'), 52),
 (('Export', '2009'), 52),
 (('Export', '2010'), 61),
 (('Export', '2011'), 55),
 (('Export', '2012'), 49),
 (('Export', '2013'), 50),
 (('Export', '2014'), 62),
 (('Export', '2015'), 55),
 (('Export', '2016'), 53),
 (('Import', '1988'), 5),
 (('Import', '1989'), 14),
 (('Import', '1990'), 26),
 (('Import', '1991'), 21),
 (('Import', '1992'), 28),
 (('Import', '1993'), 33),
 (('Import', '1994'), 47),
 (('Import', '1995'), 64),
 (('

In [9]:
#Divide a base de dados através do split tendo como parâmetro ';', em seguida retorna como chave o tipo de fluxo e o ano e como valor 
def map_tipo_ano(linha):
  colunas = linha.split(";")
  return ((colunas[flow], colunas[ano]), 1)

#Soma as ocorrências por chave
def reduce_tipo_ano (a,b):
  return a + b

#Realiza o map através da função 'map_tipo_ano' e em seguida realiza o reduce pela chave através da função reduce_tipo_ano. 
#Posteriormente, salva os resultados em um arquivo txt
def transacoes_tipo_ano(con):
  map_resultado = rdd.map(map_tipo_ano)
  qtd_tipo_ano = map_resultado.reduceByKey(reduce_tipo_ano)
  qtd_tipo_ano.saveAsTextFile("/content/output/Q3")


In [10]:
#chama a função transacoes_tipo_ano com a base de dados
transacoes_tipo_ano(sc)

# Questão 4
### O preço médio das commodities por ano

In [ ]:
#Realiza a função map e retorna o ano e o código da commodity como chave e o preço da commodity e 1 como valor
rdd_media_ano = rdd.map(lambda x: ((x.split(';')[ano], x.split(";")[commodity_code]), (float(x.split(';')[trade_usd]), 1)))

#Através do reduceByKey, soma os preços da commodity e a quantidade de 1 para cada chave
somas = rdd_media_ano.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

#Através da função mapValues divide a soma dos preços e a quantidade encontrada por chave
preço_medio_commodities_ano = somas.mapValues(lambda x: (x[0] / x[1]))

#Faz a ordenação ascendente pelas chaves retornando os dez primeiros resultados encontrados através da função take
preço_medio_commodities_ano.sortByKey(ascending=True).take(10)

[(('1988', '100110'), 1902536.5),
 (('1988', '100190'), 95262069.0),
 (('1988', '100200'), 11254924.333333334),
 (('1988', '100300'), 28633115.666666668),
 (('1988', '100400'), 15893620.666666666),
 (('1988', '100510'), 4172584.625),
 (('1988', '100590'), 14670304.75),
 (('1988', '100610'), 459162.0),
 (('1988', '100620'), 10741745.666666666),
 (('1988', '100630'), 37426167.5)]

In [ ]:
#Divide a base de dados através do split tendo como parâmetro ';', em seguida retorna como chave o ano e o código da 
#commodity e como valor o preço da commodity e 1
def map_media_ano(linha):
  colunas = linha.split(";")
  return((colunas[ano], colunas[commodity_code]), (float(colunas[trade_usd]), 1))

#Soma os preços da commodity e a quantidade de 1 para cada chave, retornando a soma dos preços e a soma dos contadores
def somas_media_ano(a,b):
  sum_preco = a[0] + b[0]
  sum_count = a[1] + b[1]
  return (sum_preco, sum_count)

#Divide a soma dos preços pela quantidade
def reduce_media_ano(a):
  return a[0] / a[1]

#Realiza o map através da função 'map_media_ano', em seguida realiza o reduce pela chave através da função somas_media_ano. 
#Depois, através da função mapValues chamando reduce_media_ano divide a soma dos preços e a quantidade encontrada por chave,
#ordenando de forma ascendente através da função sortByKey.
#Posteriormente, salva os resultados em um arquivo txt
def media_preco_ano(con):
  map_resultado = rdd.map(map_media_ano)
  reduce_resultado = map_resultado.reduceByKey(somas_media_ano)
  resultado = reduce_resultado.mapValues(reduce_media_ano).sortByKey(ascending=True)
  resultado.saveAsTextFile("/content/output/Q4")

In [ ]:
#chama a função media_preco com a base de dados
media_preco_ano(sc)

# Questão 5
### O preço médio das transações que envolvem o Brasil por tipo de unidade, ano,categoria e fluxo do tipo 'Export'

In [ ]:
#Filtra a base de dados, retornando apenas o que possui Brazil na coluna pais
rdd_brasil = rdd.filter(lambda x: x.split(';')[pais] == 'Brazil')

#Filtra a base de dados gerada através do filter acima, retornando apenas o que possui Brazil na coluna pais e Export na coluna flow
rdd_brasil_export = rdd_brasil.filter(lambda x: x.split(';')[flow] == 'Export')

In [ ]:
#Realiza a função map e retorna o ano, código da commodity, tipo de unidade e a quantidade como chave, e o preço da commodity e 1 como valor
rdd_preco_medio_brazil_export = rdd_brasil_export.map(lambda x: ((x.split(';')[ano], x.split(';')[commodity_code], x.split(';')[quantity_name], x.split(';')[quantity]), (float(x.split(';')[trade_usd]), 1)))

#Através do reduceByKey, soma os preços da commodity e a quantidade de 1 para cada chave
somas = rdd_preco_medio_brazil_export.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

#Através da função mapValues divide a soma dos preços e a quantidade encontrada por chave
preço_medio_commodities_ano = somas.mapValues(lambda x: (x[0] / x[1]))

#Faz a ordenação ascendente pelas chaves retornando os dez primeiros resultados encontrados através da função take
preço_medio_commodities_ano.sortByKey(ascending=True).take(10)

[(('1989', '100200', 'Weight in kilograms', '57601.0'), 24600.0),
 (('1989', '100300', 'Weight in kilograms', '28500.0'), 12029.0),
 (('1989', '100810', 'Weight in kilograms', '3396187.0'), 586041.0),
 (('1989', '100890', 'Weight in kilograms', '635000.0'), 95250.0),
 (('1989', '10310', 'Weight in kilograms', '24753.0'), 14150.0),
 (('1989', '10420', 'Weight in kilograms', '1515.0'), 2137.0),
 (('1989', '10511', 'Weight in kilograms', '42302.0'), 1249136.0),
 (('1989', '10600', 'Weight in kilograms', '20457.0'), 92071.0),
 (('1989', '110312', 'Weight in kilograms', '4277.0'), 4759.0),
 (('1989', '110811', 'Weight in kilograms', '1000.0'), 2447.0)]

In [ ]:
#Divide a base de dados através do split tendo como parâmetro ';', em seguida retorna ano, código da commodity,
#tipo de unidade  e a quantidade como chave e o preço da commodity e 1 como valor
def map_brasil_export (linha):
  colunas = linha.split(';')
  return ((colunas[ano], colunas[commodity_code], colunas[quantity_name], colunas[quantity]), (float(colunas[trade_usd]), 1))

#Soma os preços da commodity e a quantidade de 1 para cada chave, retornando a soma dos preços e a soma dos contadores
def somas_brasil_export(a,b):
  sum_preco = a[0] + b[0]
  sum_count = a[1] + b[1]
  return (sum_preco, sum_count)

#Divide a soma dos preços pela quantidade
def reduce_brasil_export(a):
  return a[0] / a[1]

#Realiza o map através da função 'map_brasil_export', em seguida realiza o reduce pela chave através da função somas_brasil_export. 
#Depois, através da função mapValues chamando reduce_media_ano divide a soma dos preços e a quantidade encontrada por chave,
#ordenando de forma ascendente pela função sortByKey.
#Posteriormente, salva os resultados em um arquivo txt
def media_brasil_export(con):
  map_resultado = rdd_brasil_export.map(map_brasil_export)
  reduce_resultado = map_resultado.reduceByKey(somas_brasil_export)
  resultado = reduce_resultado.mapValues(reduce_media_ano).sortByKey(ascending=True)
  resultado.saveAsTextFile("/content/output/Q5")

In [ ]:
#chama a função media_preco com a base de dados
media_brasil_export(sc)

# Questão 6
### Preço máximo, mínimo e média dos preços por tipo de unidade e ano  

In [ ]:
#Divide a base de dados através do split tendo como parâmetro ';', em seguida retorna ano e o tipo de unidade como chave,
#e o preço da commodity três vezes e 1 como valor
def map_max_min_media(linha):
  colunas = linha.split(";")
  return ((colunas[ano], colunas[quantity_name]), (float(colunas[trade_usd]), float(colunas[trade_usd]), float(colunas[trade_usd]), 1))

#Utilizando o primeiro valor dos valores, através da função max, econtra o maior preço,
#utilizando o segundo valor dos valores, através da função min, encontra o menor preço.
#Soma os preços da commodity, utilizando o terceiro valor dos valores, e a quantidade de 1 para cada chave,
#utilizando o quarto valor dos valores, retornando o preço máximo, preço mínimo, a soma dos preços e a soma dos contadores
def somas_media(a,b):
  max_preco = max([a[0],b[0]])
  min_preco = min([a[1],b[1]])

  sum_preco = a[2] + b[2]
  sum_count = a[3] + b[3]

  return (max_preco, min_preco, sum_preco, sum_count)

#Divide a soma dos preços pela quantidade, retorna o preço máximo(posição 0), preço mínimo(posição 1) e a média
def reduce_min_max_media(a):
  media = a[2] / a[3]

  return (a[0], a[1], media)

#Realiza o map através da função 'map_max_min_media', em seguida realiza o reduce pela chave através da função somas_media. 
#Depois, através da função mapValues chamando reduce_min_max_media divide a soma dos preços e a quantidade encontrada por chave,
#retornando preço máximo(posição 0), preço mínimo(posição 1) e a média, ordenando de forma ascendente pela função sortByKey.
#Posteriormente, salva os resultados em um arquivo txt
def media_max_min(con):
  map_resultado = rdd.map(map_max_min_media)
  reduce_resultado = map_resultado.reduceByKey(somas_media)
  resultado = reduce_resultado.mapValues(reduce_min_max_media).sortByKey(ascending=True)
  resultado.saveAsTextFile("/content/output/Q6")

In [ ]:
#chama a função media_max-min com a base de dados
media_max_min(sc)

# Questão 7
### A commodity mais comercializada em 2016 por tipo de fluxo

In [8]:
rdd_filter = rdd.filter(lambda x: x.split(';')[ano] == '2016')

def map_commodity(x):
  colunas = x.split(";")
  return ((colunas[2], colunas[4]), colunas[8])

def map_2(x):
  key = x[0]
  value = x[1]
  return (key[1], (key[0], value))

def reduce_2(x, y):
  result = None
  if x[1] > y[1]:
    result = x
  else:
    result = y
  return result

rdd_map = rdd_filter.map(map_commodity)
rdd_reduce_key = rdd_map.reduceByKey(lambda x, y: x + y)

rdd_map2 = rdd_reduce_key.map(map_2)
rdd_reduce_key = rdd_map2.reduceByKey(reduce_2)

rdd_reduce_key.collect()

[('Export', ('440710', '948489.0')),
 ('Import', ('321511', '6472.0')),
 ('Re-Export', ('902720', '39.0')),
 ('Re-Import', ('280110', '420.0'))]